# Phase 6 — Calibration & Validation du modèle PD

## Objectifs
1) Mesurer la calibration : la PD prédite correspond-elle aux défauts observés ?
2) Produire des métriques robustes : Brier score + courbe de calibration
3) (Simple) Tester un stress test : impact d'un scénario défavorable sur la PD moyenne

## Livrables
- `reports/tableau_exports/06_calibration_bins.csv`
- `reports/tableau_exports/06_brier_score.csv`
- `reports/tableau_exports/06_stress_test_summary.csv`


In [22]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import brier_score_loss

In [9]:
X = pd.read_csv("data/processed/X_features.csv")
y = pd.read_csv("data/processed/y_target.csv").squeeze()

X.shape, y.shape

((250000, 10), (250000,))

In [11]:
#scaler fontion 
scaler = StandardScaler()

In [14]:
#split data into train and test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42, stratify = y)

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

In [16]:
# logistic regression model with regularisation (l2)

model = LogisticRegression(
    penalty="l2",
    C=1.0,
    solver="lbfgs",
    max_iter=1000
)
model.fit(X_train_scaled, y_train)

LogisticRegression(max_iter=1000)

In [21]:
# probabilitie of default PD
y_proba = model.predict_proba(X_test_scaled)[:,1]
y_proba[:5]


array([0.23693699, 0.30098   , 0.37173634, 0.20773532, 0.04972834])

In [23]:
# calibration using brier score 

brier = brier_score_loss(y_test, y_proba)
brier

np.float64(0.14692392308123176)

Le Brier score obtenu est de 0.147, indiquant une calibration correcte du modèle.
Les probabilités de défaut estimées sont globalement cohérentes avec les fréquences observées, bien qu’une amélioration de la calibration soit possible

In [25]:
brier_df = pd.DataFrame([{"brier_score": brier}])
brier_df.to_csv("reports/tableau_exports/06_brier_score.csv", index=False)
brier_df


,brier_score
0,0.146924


#### Courbe de calibration (par bins)
- Concept : Calibration curve
  
On regroupe les prêts par tranches de PD prédite, et on compare :

- PD moyenne prédite
- taux de défaut observé

##### Si le modèle est bien calibré :
les deux sont proches. `pd_mean` ≈ `default_rate`

In [27]:
calib = pd.DataFrame({"y_true": y_test.values, "pd_pred": y_proba})

# 10 bins = 10 groupes de risque
calib["bin"] = pd.qcut(calib["pd_pred"], q=10, duplicates="drop")

calib_bins = calib.groupby("bin", observed=True).agg(
    n=("y_true", "size"),
    pd_mean=("pd_pred", "mean"),
    default_rate=("y_true", "mean")
).reset_index()

calib_bins


,bin,n,pd_mean,default_rate
0,"(0.0235, 0.0614]",5000,0.052494,0.048800
1,"(0.0614, 0.105]",5000,0.074179,0.075200
2,"(0.105, 0.13]",5001,0.120103,0.110178
3,"(0.13, 0.146]",4999,0.137523,0.138828
4,"(0.146, 0.186]",5000,0.163954,0.160000
5,"(0.186, 0.21]",5000,0.198926,0.202000
6,"(0.21, 0.248]",5000,0.225228,0.233600
7,"(0.248, 0.292]",5000,0.272664,0.277800
8,"(0.292, 0.366]",5000,0.321288,0.323600
9,"(0.366, 0.647]",5000,0.429455,0.425800


Le modèle est globalement bien calibré.
Pour chaque décile de risque, la probabilité moyenne prédite est très proche du taux de défaut observé.
Cela signifie que lorsque le modèle annonce une PD de X %, cette valeur est cohérente avec la réalité empirique.

Le modèle discrimine correctement ET attribue des probabilités fiables.

In [28]:
calib_bins.to_csv("reports/tableau_exports/06_calibration_bins.csv", index=False)
print("Export OK: reports/tableau_exports/06_calibration_bins.csv")


Export OK: reports/tableau_exports/06_calibration_bins.csv


#### Stress test simple (scénario défavorable)

Un stress test = “si les conditions se dégradent, que devient la PD ?”

scénario : la durée augmente de 12 mois

Comme `term_months` est une feature, on simule ce choc.

In [39]:
X_test_stress = X_test.copy()
X_test_stress["term_months"] = X_test_stress["term_months"] + 12

X_test_stress_scaled = scaler.transform(X_test_stress)
pd_stress = model.predict_proba(X_test_stress_scaled)[:, 1]

summary = pd.DataFrame([
    {"scenario": "baseline", "pd_mean": float(np.mean(y_proba))},
    {"scenario": "stress_term_plus_12", "pd_mean": float(np.mean(pd_stress))}
])

summary


,scenario,pd_mean
0,baseline,0.199581
1,stress_term_plus_12,0.193260


### Interprétation du stress sur la maturité

Le stress consistant à augmenter la maturité des prêts de 12 mois entraîne une baisse de la PD moyenne du portefeuille.
Ce résultat s’explique par un effet de sélection : dans les données historiques, les durées plus longues sont principalement
associées à des profils emprunteurs plus solides.
Le modèle capture donc une relation statistique et non causale entre maturité et risque.
Ce résultat souligne l’importance d’interpréter les stress tests comme des analyses de sensibilité du modèle,
et non comme des prédictions causales directes.

Nb : Un stress test n’est pas une preuve causale.
C’est un outil d’exploration du comportement du modèle.


In [41]:
summary.to_csv("reports/tableau_exports/06_stress_test_summary.csv", index=False)
print("Export OK: reports/tableau_exports/06_stress_test_summary.csv")


Export OK: reports/tableau_exports/06_stress_test_summary.csv
